# Analyse de la Répartition des Équipements Sportifs dans le Val-d'Oise

Le département du Val-d'Oise, situé dans la région Île-de-France, dispose d'un patrimoine riche en équipements sportifs. L'analyse de ces équipements est essentielle pour comprendre les besoins et les disparités en matière d'infrastructures sportives au sein des communes. Les données utilisées proviennent du site data.iledefrance.fr, qui met à disposition le recensement des équipements sportifs dans le Val-d'Oise sous format CSV. L'URL source est : http://data.iledefrance.fr/explore/dataset/recensement_des_equipements_sportifs_dans_le_val-d_oise/download?format=csv.

L'objectif de cette analyse est de compter le nombre d'équipements sportifs par commune en utilisant le code INSEE ("new_code"). Cela permettra d'avoir une vision claire de la répartition de ces équipements et d'identifier les communes qui en sont les mieux pourvues ou qui pourraient nécessiter un renforcement de leurs infrastructures sportives.


## Méthodologie

La méthodologie utilisée pour cette analyse repose sur une requête SQL exécutée via DuckDB, un outil d'analyse de données en colonnes. La requête a consisté à regrouper les équipements sportifs par leur code INSEE ("new_code"), à compter le nombre d'équipements pour chaque groupe, et à ordonner les résultats par code INSEE. Les résultats ont ensuite été utilisés pour créer une carte choropleth représentant la répartition des équipements sportifs par commune.


Les données ont été enrichies avec des données géospatiales provenant d'un fichier GeoJSON contenant les contours des communes françaises. L'utilisation de bibliothèques telles que Pandas, Geopandas, et Plotly Express a permis de manipuler et de visualiser efficacement les données.


Ainsi, cette analyse propose une visualisation claire et interactive de la répartition des équipements sportifs dans le Val-d'Oise, permettant une meilleure compréhension des disparités géographiques et facilitant la prise de décision pour les acteurs locaux.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("http://data.iledefrance.fr/explore/dataset/recensement_des_equipements_sportifs_dans_le_val-d_oise/download?format=csv", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT 
  "new_code" AS code_insee,
  COUNT(*) AS nombre_equipements
FROM 
  loaded_dataset
GROUP BY 
  "new_code"
ORDER BY 
  "new_code" """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui permet de créer des visualisations interactives et personnalisables. Cette technologie est adaptée car elle offre une grande flexibilité pour représenter des données géospatiales, comme ici une carte choroplèthe représentant la répartition des équipements sportifs par commune. Cela permet une exploration interactive des données.

In [3]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import geopandas as gpd
import plotly.express as px
import requests
import io

url_communes = 'https://france-geojson.gregoiredavid.fr/repo/communes.geojson'
communes_geojson = requests.get(url_communes).content
communes = gpd.read_file(io.BytesIO(communes_geojson))
communes["code"] = communes["code"].astype(str)
df["code_insee"] = df["code_insee"].astype(str)

carto = communes.merge(df, left_on="code", right_on="code_insee")

carto = carto[~carto["nombre_equipements"].isna()]

carto["geometry"] = carto["geometry"].simplify(0.01)

fig = px.choropleth_mapbox(
    carto,
    geojson=carto.geometry,
    locations=carto.index,
    color="nombre_equipements",
    color_continuous_scale="YlGn",
    range_color=[carto["nombre_equipements"].min(), carto["nombre_equipements"].max()],
    mapbox_style="carto-positron",
    center={"lat": carto.centroid.y.mean(), "lon": carto.centroid.x.mean()},
    zoom=9.2,
    hover_data={"nom": True, "code": True, "nombre_equipements": True},
    labels={"nombre_equipements": "Équipements", "nom": "Commune", "code": "Code INSEE"},
    width=900,
    height=600
)

fig.update_layout(
    title_text="Répartition des équipements sportifs par commune",
    title_font_size=17,
    title_font_family="Segoe UI",
    margin=dict(l=0, r=0, t=35, b=0),
    coloraxis_colorbar_title_text="Nb Équip."
)

dataviz = fig
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*